In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Problem 1

In [2]:
# Classical Brownian Motion

# Assume Pt-1 equal to 10
def CBM(Pt_1, samples):

    # Assume the variance of rt equal to 1
    rt = np.random.randn(samples)
    Pt = Pt_1 + rt
    Ex_Pt = np.mean(Pt)
    Std_Pt = np.std(Pt)
    Analytical_Ex = Pt_1
    Analytical_Std = 1
    print(f"Numerical Results: Mean: {Ex_Pt}, Std: {Std_Pt}; Analytical Results: Mean: {Analytical_Ex}, Std: {Analytical_Std}")
    return

CBM(10,100000)
# t_value = (Ex_Pt - Pt_1)/(np.std(Pt) / np.sqrt(1000))
# p_value = 2 * (1 - t.cdf(abs(t_value), 1000-1))
# print(t_value, p_value)

Numerical Results: Mean: 10.001157206243105, Std: 1.0007145036971405; Analytical Results: Mean: 10, Std: 1


In [3]:
# Arithmetic Return System

# Assume Pt-1 equal to 10
def ARS(Pt_1, samples):

    # Assume the variance of rt equal to 1
    rt = np.random.randn(samples)
    Pt = Pt_1*(1 + rt)
    Ex_Pt = np.mean(Pt)
    Std_Pt = np.std(Pt)
    Analytical_Ex = Pt_1
    Analytical_Std = np.sqrt(Pt_1**2*1)
    print(f"Numerical Results: Mean: {Ex_Pt}, Std: {Std_Pt}; Analytical Results: Mean: {Analytical_Ex}, Std: {Analytical_Std}")
    return

ARS(10,100000)

Numerical Results: Mean: 10.05619706928659, Std: 10.00267481854725; Analytical Results: Mean: 10, Std: 10.0


In [4]:
# Log Return or Geometric Brownian Motion

# Assume Pt-1 equal to 10

def LRS(Pt_1, samples):
    # Assume the variance of rt equal to 1
    rt = np.random.randn(samples)
    Pt = Pt_1*np.exp(rt)
    Ex_Pt = np.mean(Pt)
    Std_Pt = np.std(Pt)
    Analytical_Ex = Pt_1 * np.exp(0.5)
    Analytical_Std = np.sqrt(Pt_1**2 * (np.exp(2)-np.exp(1)))
    print(f"Numerical Results: Mean: {Ex_Pt}, Std: {Std_Pt}; Analytical Results: Mean: {Analytical_Ex}, Std: {Analytical_Std}")
    return
    
LRS(10,100000)

Numerical Results: Mean: 16.533824573311836, Std: 21.454134548569236; Analytical Results: Mean: 16.487212707001284, Std: 21.61197415895088


# Problem 2

In [5]:
# Data Preprocess
def return_calculate(prices, method="DISCRETE", date_column="Date"):
    # Make sure date column exist
    if date_column not in prices.columns:
        raise ValueError(f"dateColumn: {date_column} not in DataFrame")

    # Choose all colums excpet for date
    cols = [col for col in prices.columns if col != date_column]
    
    # Extract Price data
    p = prices[cols].values
    n, m = p.shape
    
    # Calculate price ratios at consecutive points in time
    p2 = p[1:, :] / p[:-1, :]
    
    # Calculate rate of return based on method
    if method.upper() == "DISCRETE":
        p2 -= 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\", \"DISCRETE\")")
    
    # Create a DataFrame containing the results
    out = pd.DataFrame(p2, columns=cols)
    out[date_column] = prices[date_column].values[1:]
    
    return out

def ewCovar(x, lbda):
    m, n = x.shape
    w = np.empty(m)
    # Remove the mean from the series
    xm = np.mean(x, axis=0)
    x = (x - xm).values
    # Calculate weight. Realize we are going from oldest to newest
    w = (1 - lbda) * lbda ** np.arange(m)[::-1]
    # Normalize weights to 1
    w /= np.sum(w)
    w = w.reshape(-1, 1)

#     print(w.shape)
#     print(x.shape)
    # covariance[i,j] = (w * x.T) @ x
    return (w * x).T @ x

In [6]:
# Read data and calculate return
df_price = pd.read_csv("DailyPrices.csv")
current_prices = df_price.iloc[-1]
df_ret = return_calculate(df_price, method = "DISCRETE")

df_ret

,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,META,...,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT,Date
0,-0.010544,-0.013611,-0.016667,-0.002425,-0.020808,-0.017223,-0.025076,-0.016915,-0.016854,-0.030479,...,-0.010593,0.033107,-0.010428,-0.019242,-0.004236,-0.015244,0.001846,-0.012198,-0.026355,2022-09-02
1,-0.003773,-0.008215,-0.010974,-0.010980,-0.013336,-0.009643,0.015581,-0.011042,-0.003890,-0.011103,...,0.008449,-0.014118,0.000572,0.001848,-0.008019,-0.000892,-0.012695,-0.002717,0.013275,2022-09-06
2,0.017965,0.009254,0.019111,0.026723,0.018795,0.024717,0.033817,0.027912,0.016089,0.011669,...,0.020295,-0.008030,0.038537,0.018731,0.012279,0.022698,0.008503,0.026994,0.020930,2022-09-07
3,0.006536,-0.009618,0.001666,0.002626,0.020126,-0.009776,0.019598,-0.009595,0.008184,0.010412,...,0.013945,0.029951,0.015880,0.019083,0.016574,-0.011908,0.026116,0.029901,0.008362,2022-09-08
4,0.015535,0.018840,0.022977,0.026575,0.028377,0.020945,0.036023,0.021568,0.008576,0.043749,...,0.017244,0.038774,-0.004179,0.018863,0.026460,0.036721,0.015431,0.005385,-0.000306,2022-09-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,0.000586,0.016913,-0.003513,-0.002920,0.001503,0.005895,-0.033201,0.004772,0.006986,0.007459,...,0.006938,0.010399,0.013118,-0.006183,0.020125,-0.003329,-0.001639,0.001890,-0.003386,2023-09-18
261,-0.002074,0.006181,-0.001246,-0.016788,-0.010144,-0.001230,0.004599,-0.000936,0.000135,0.008329,...,-0.013644,-0.012743,0.013589,-0.002247,-0.016519,0.012970,0.000938,0.000566,-0.012087,2023-09-19
262,-0.009193,-0.019992,-0.023977,-0.017002,-0.029435,-0.031150,-0.014672,-0.030541,-0.009879,-0.017701,...,-0.006986,-0.010591,0.001544,-0.018361,-0.010062,-0.002748,-0.008903,0.020177,0.000282,2023-09-20
263,-0.016528,-0.008889,-0.003866,-0.044053,-0.028931,-0.024675,-0.026239,-0.023999,-0.009651,-0.013148,...,-0.018635,-0.016223,-0.002032,-0.011646,-0.013686,-0.026725,-0.013948,-0.002403,-0.045601,2023-09-21


In [7]:
holdings = {
    "META": 1
}

# Select Data
name = list(set(df_price.columns) & set(holdings.keys()))
current_prices = current_prices[name]
returns = df_ret[name]
returns = returns - np.mean(returns, axis = 0)

PV = 0.0
Asset_value = []
for nm in name:
    value = holdings[nm] * current_prices[nm]
    PV += value
    Asset_value.append(value)

def VaR_cal(method, ret, PV, Asset_value, holdings, name, current_prices, alpha):

    # Calcualte Covariance Matrix and Portfiolio Volaitility
    if method == "Normal":
        # R_gradients also equal to weights
        R_gradients = np.array(Asset_value) / PV
        Sigma = np.cov(ret, rowvar=False)
        p_sig = np.sqrt(np.dot(R_gradients.T, np.dot(Sigma, R_gradients)))
        VaR = (-PV) * norm.ppf(alpha) * p_sig
    
    elif method == "EW_Normal":
        R_gradients = np.array(Asset_value) / PV
        Sigma = ewCovar(ret,0.94)
        p_sig = np.sqrt(np.dot(R_gradients.T, np.dot(Sigma, R_gradients)))
        VaR = (-PV) * norm.ppf(alpha) * p_sig
    
    elif method == "MLE_T":
        params = stats.t.fit(ret)
        df, loc, scale = params
        VaR = (-PV) * stats.t.ppf(alpha, df, loc, scale)
    
    elif method == "AR_1":
        model = ARIMA(ret, order=(1, 0, 0))
        model_fit = model.fit()
        phi_0 = model_fit.params['const']  # or model_fit.params[0]
        phi_1 = model_fit.params['ar.L1']  # or model_fit.params[1]
        predicted_return = phi_0 + phi_1 * ret.values[-1,0]
        
        # Calculate Std and VaR
        residual_std = model_fit.resid.std()
        VaR = (-PV) * (predicted_return + norm.ppf(alpha) * residual_std)
    
    elif method == "Historical":
        rand_indices = np.random.choice(ret.shape[0], size=10000, replace=True)
        sim_ret = ret.values[rand_indices, :]
        sim_price = current_prices.values * (1 + sim_ret)
        vHoldings = np.array([holdings[nm] for nm in name])
        pVals = sim_price @ vHoldings
        VaR = PV - np.percentile(pVals, alpha * 100)
    return VaR
#     print(f"{method} VaR")
#     print(f"Current Portfolio Value: {PV}")
#     print(f"Current Portfolio VaR: {VaR}")

mtd_list = ["Normal","EW_Normal","MLE_T","AR_1","Historical"]
mtd_dic = {}
for mtd in mtd_list:
    VaR = VaR_cal(mtd, returns, PV, Asset_value, holdings, name, current_prices, 0.05)
    mtd_dic[mtd] = [VaR, VaR/(-PV)]

# Result
pd.DataFrame(mtd_dic, index = ["VaR $", "VaR %"])

,Normal,EW_Normal,MLE_T,AR_1,Historical
VaR $,16.236135,8.966972,12.900730,16.101476,11.815958
VaR %,-0.054287,-0.029982,-0.043135,-0.053837,-0.039508


# Problem3

In [8]:
df_port = pd.read_csv("portfolio.csv")

In [9]:
VaR_dic = {}
port_list = ["A", "B", "C", "Total"]

def VaR_port_data(df_port, df_price, port, df_ret):
    if port == "Total":
        port = ["A","B","C"]
        
    name = df_port[df_port["Portfolio"].isin(port) if isinstance(port, list) else df_port["Portfolio"] == port]["Stock"].values
    holdings = {}
    for nm in name:
        holdings[nm] = df_port[(df_port["Portfolio"].isin(port) if isinstance(port, list) else df_port["Portfolio"] == port) & (df_port["Stock"] == nm)]["Holding"].values[0]
    current_prices = df_price.iloc[-1]
    current_prices = current_prices[name]
    returns = df_ret[name]
    returns = returns - np.mean(returns, axis = 0)
    PV = 0.0
    Asset_value = []

    for nm in name:
        value = holdings[nm] * current_prices[nm]
        PV += value
        Asset_value.append(value)
    return returns, PV, Asset_value, holdings, name, current_prices

for port in port_list:
    returns, PV, Asset_value, holdings, name, current_prices = VaR_port_data(df_port, df_price, port, df_ret)
    VaR_dic[port] = [VaR_cal("EW_Normal", returns, PV, Asset_value, holdings, name, current_prices, 0.05)]

# Result 
pd.DataFrame(VaR_dic, index = ["VaR"])

,A,B,C,Total
VaR,15426.968017,8082.572402,18163.291619,38941.375729


In [10]:
# Choose Log Return
df_ret = return_calculate(df_price, method = "LOG")

for port in port_list:
    returns, PV, Asset_value, holdings, name, current_prices = VaR_port_data(df_port, df_price, port, df_ret)
    VaR_dic[port] = [VaR_cal("EW_Normal", returns, PV, Asset_value, holdings, name, current_prices, 0.05)]

pd.DataFrame(VaR_dic, index = ["VaR"])

,A,B,C,Total
VaR,15433.515096,8089.616241,18081.612831,38904.842412
